In [2]:
import os
import math
import random

import numpy as np
import torch
import torch.nn as nn
import h5py
import pandas as pd
from tqdm import tqdm

# ============================================================
# Config  (EDIT THESE PATHS IF NEEDED)
# ============================================================

def find_root_BH_datasets(h5file):
    """
    Fallback: find B and H datasets directly under the root (no groups).
    We pick keys that contain 'b' (but not 'db') and 'h' (but not 'dh').
    """
    keys = list(h5file.keys())
    low = {k: k.lower() for k in keys}

    B_cand = [k for k in keys if "b" in low[k] and "db" not in low[k]]
    H_cand = [k for k in keys if "h" in low[k] and "dh" not in low[k]]

    if not B_cand or not H_cand:
        raise RuntimeError(f"Cannot identify B/H datasets at root. Keys={keys}")

    # Just pick the first ones in sorted order
    B_key = sorted(B_cand)[0]
    H_key = sorted(H_cand)[0]

    print(f"   Using ROOT datasets: B='{B_key}', H='{H_key}'")
    return h5file[B_key], h5file[H_key]


DEVICE = torch.device("cpu")
print("Device:", DEVICE)

TRAIN_ROOT   = "/Users/Ankan/Desktop/Magnet_Challenge_2/Training Data/78/Training Data"
PRETEST_ROOT = "/Users/Ankan/Desktop/Magnet_Challenge_2/Training Data/78/Pretest"
MODEL_PATH   = "/Users/Ankan/Desktop/Magnet_Challenge_2/Training Data/78/Training Data/global_lstm_model.pt"

TRAIN_MATERIALS   = ["3C90", "3C94", "3E6", "3F4", "77", "78",
                     "N27", "N30", "N49", "N87"]

PRETEST_MATERIALS = ["3C90", "3C94", "3E6", "3F4", "77", "78",
                     "N27", "N30", "N49", "N87", "N87-2"]

FREQS      = [1, 2, 3, 4, 5, 6, 7]
SEQ_LEN    = 80
STRIDE     = 5

# we’ll treat all pretest excitations as “freq 4” embedding for now
FREQ_FOR_PRETEST = 4
FREQ_TO_IDX = {f: i for i, f in enumerate(FREQS)}


# ============================================================
# Helpers
# ============================================================

def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


def load_1d_csv(path: str) -> np.ndarray:
    arr = pd.read_csv(path, header=None).values.squeeze()
    return np.asarray(arr, dtype=np.float32).reshape(-1)


# ============================================================
# Global model (same as training-time architecture)
# ============================================================

class GlobalLSTM(nn.Module):
    """
    Matches the trained global model:
    - LSTM over [B, H, dB/dt]
    - material embedding + freq embedding concatenated to last hidden state
    - fc sees hidden_dim + mat_emb_dim + freq_emb_dim = 38
    """
    def __init__(
        self,
        n_materials: int,
        n_freqs: int,
        input_dim: int = 3,
        hidden_dim: int = 32,
        mat_emb_dim: int = 4,   # <<< MUST BE 4 (from checkpoint)
        freq_emb_dim: int = 2,  # <<< MUST BE 2 (from checkpoint)
        num_layers: int = 1,
    ):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
        )
        self.mat_emb = nn.Embedding(n_materials, mat_emb_dim)
        self.freq_emb = nn.Embedding(n_freqs, freq_emb_dim)

        # 32 (hidden) + 4 (mat emb) + 2 (freq emb) = 38
        self.fc = nn.Linear(hidden_dim + mat_emb_dim + freq_emb_dim, 1)

    def forward(self, x, mat_idx, freq_idx):
        # x: (B, L, 3)
        out, _ = self.lstm(x)          # (B, L, hidden_dim)
        last = out[:, -1, :]           # (B, hidden_dim)

        m_vec = self.mat_emb(mat_idx)  # (B, mat_emb_dim)
        f_vec = self.freq_emb(freq_idx)# (B, freq_emb_dim)

        feat = torch.cat([last, m_vec, f_vec], dim=-1)  # (B, hidden_dim+embs)
        y = self.fc(feat).squeeze(-1)
        return y


# ============================================================
# Normalisation from training CSVs
# ============================================================

def compute_global_normalisation(root, materials, freqs):
    all_B, all_H, all_dB = [], [], []

    print("\n📌 Computing global normalisation stats from training CSVs...")
    for mat in tqdm(materials, desc="Materials"):
        mat_dir = os.path.join(root, mat)
        if not os.path.isdir(mat_dir):
            continue

        for f in freqs:
            B_path = os.path.join(mat_dir, f"{mat}_{f}_B.csv")
            H_path = os.path.join(mat_dir, f"{mat}_{f}_H.csv")
            T_path = os.path.join(mat_dir, f"{mat}_{f}_T.csv")

            if not (os.path.exists(B_path) and os.path.exists(H_path) and os.path.exists(T_path)):
                continue

            B = load_1d_csv(B_path)
            H = load_1d_csv(H_path)
            T = load_1d_csv(T_path)

            n = min(len(B), len(H), len(T))
            B, H = B[:n], H[:n]
            dBdt = np.gradient(B)

            all_B.append(B)
            all_H.append(H)
            all_dB.append(dBdt)

    B_cat = np.concatenate(all_B)
    H_cat = np.concatenate(all_H)
    dB_cat = np.concatenate(all_dB)

    stats = {
        "B_mean": float(B_cat.mean()),
        "B_std":  float(B_cat.std() + 1e-8),
        "H_mean": float(H_cat.mean()),
        "H_std":  float(H_cat.std() + 1e-8),
        "dB_mean": float(dB_cat.mean()),
        "dB_std":  float(dB_cat.std() + 1e-8),
    }

    print("\nGlobal stats:")
    for k, v in stats.items():
        print(f"  {k}: {v:.6f}")

    return stats


# ============================================================
# Build windows
# ============================================================

def build_windows_from_segment(B, H, stats, seq_len=80, stride=5):
    B = B.astype(np.float32)
    H = H.astype(np.float32)
    n = min(len(B), len(H))
    B, H = B[:n], H[:n]

    dBdt = np.gradient(B).astype(np.float32)

    Bn  = (B  - stats["B_mean"])   / stats["B_std"]
    Hn  = (H  - stats["H_mean"])   / stats["H_std"]
    dBn = (dBdt - stats["dB_mean"]) / stats["dB_std"]

    inputs, targets = [], []

    for start in range(0, n - seq_len, stride):
        end = start + seq_len
        if end >= n:
            break

        x = np.stack([Bn[start:end], Hn[start:end], dBn[start:end]], axis=-1)
        y = Hn[end]
        inputs.append(x)
        targets.append(y)

    if not inputs:
        return None, None

    return np.stack(inputs), np.stack(targets)


# ============================================================
# Identify B & H datasets inside h5
# ============================================================

def find_BH_datasets(h5file):
    keys = list(h5file.keys())
    low = {k: k.lower() for k in keys}

    B_cand = [k for k in keys if "b" in low[k] and "db" not in low[k]]
    H_cand = [k for k in keys if "h" in low[k] and "dh" not in low[k]]

    if not B_cand or not H_cand:
        raise RuntimeError(f"Cannot identify B/H datasets. Keys={keys}")

    B_key = sorted(B_cand)[0]
    H_key = sorted(H_cand)[0]

    print(f"   Using datasets: B='{B_key}', H='{H_key}'")
    return h5file[B_key], h5file[H_key]


# ============================================================
# Pretest evaluation
# ============================================================

def evaluate_pretest_material(material, model, stats):
    true_path = os.path.join(
        PRETEST_ROOT, material, f"{material}_Testing_True.h5"
    )

    if not os.path.exists(true_path):
        print(f"[{material}] Missing Testing_True.h5 — skipping")
        return []

    print(f"\n=== Evaluating PRETEST for {material} ===")
    rows = []

    with h5py.File(true_path, "r") as f:
        # 1) detect any groups at root
        group_names = [k for k in f.keys() if isinstance(f[k], h5py.Group)]
        group_names = sorted(group_names)
        print(f"Found groups: {group_names}")

        # Determine material index (for embedding); default to 0 if not found
        if material in TRAIN_MATERIALS:
            mat_idx_val = TRAIN_MATERIALS.index(material)
        else:
            mat_idx_val = 0

        # -----------------------------
        # CASE A: grouped layout (Dataset_1, Dataset_2, ...)
        # -----------------------------
        if group_names:
            for gname in tqdm(group_names, desc=f"{material} groups"):
                grp = f[gname]
                if "B" not in grp or "H" not in grp:
                    print(f"  Skipping group {gname}: missing B/H")
                    continue

                B_seg = np.array(grp["B"][:], dtype=np.float32)
                H_seg = np.array(grp["H"][:], dtype=np.float32)

                X_norm, y_norm = build_windows_from_segment(
                    B_seg, H_seg, stats, seq_len=SEQ_LEN, stride=STRIDE
                )
                if X_norm is None:
                    continue

                X = torch.from_numpy(X_norm)
                y = torch.from_numpy(y_norm)

                # dummy freq index (we don't know actual frequency in pretest)
                mat_idx = torch.full((len(X),), mat_idx_val, dtype=torch.long)
                freq_idx = torch.zeros(len(X), dtype=torch.long)

                with torch.no_grad():
                    preds = model(X, mat_idx, freq_idx).numpy()

                diff_phys = (preds - y.numpy()) * stats["H_std"]
                rmse = float(np.sqrt(np.mean(diff_phys ** 2)))

                rms_meas = float(np.sqrt(np.mean((y.numpy() * stats["H_std"]) ** 2)))
                rel_err = rmse / (rms_meas + 1e-12) * 100.0

                rows.append({
                    "material": material,
                    "segment": gname,
                    "n_windows": len(X_norm),
                    "rmse": rmse,
                    "rel_err": rel_err,
                })

        # -----------------------------
        # CASE B: no groups → root datasets behave like before
        # -----------------------------
        else:
            try:
                B_ds, H_ds = find_root_BH_datasets(f)
            except RuntimeError as e:
                print(f"  {e} — skipping {material}")
                return rows

            shape = B_ds.shape
            if len(shape) == 1:
                # Single long sequence
                B_seg = np.array(B_ds[:], dtype=np.float32)
                H_seg = np.array(H_ds[:], dtype=np.float32)

                X_norm, y_norm = build_windows_from_segment(
                    B_seg, H_seg, stats, seq_len=SEQ_LEN, stride=STRIDE
                )
                if X_norm is not None:
                    X = torch.from_numpy(X_norm)
                    y = torch.from_numpy(y_norm)

                    mat_idx = torch.full((len(X),), mat_idx_val, dtype=torch.long)
                    freq_idx = torch.zeros(len(X), dtype=torch.long)

                    with torch.no_grad():
                        preds = model(X, mat_idx, freq_idx).numpy()

                    diff_phys = (preds - y.numpy()) * stats["H_std"]
                    rmse = float(np.sqrt(np.mean(diff_phys ** 2)))

                    rms_meas = float(np.sqrt(np.mean((y.numpy() * stats["H_std"]) ** 2)))
                    rel_err = rmse / (rms_meas + 1e-12) * 100.0

                    rows.append({
                        "material": material,
                        "segment": "root",
                        "n_windows": len(X_norm),
                        "rmse": rmse,
                        "rel_err": rel_err,
                    })

            elif len(shape) == 2:
                # Treat axis 0 as segments: (n_segments, seq_len)
                n_segments = shape[0]
                for seg_idx in tqdm(range(n_segments),
                                    desc=f"{material} segments (root)"):
                    B_seg = np.array(B_ds[seg_idx], dtype=np.float32)
                    H_seg = np.array(H_ds[seg_idx], dtype=np.float32)

                    X_norm, y_norm = build_windows_from_segment(
                        B_seg, H_seg, stats, seq_len=SEQ_LEN, stride=STRIDE
                    )
                    if X_norm is None:
                        continue

                    X = torch.from_numpy(X_norm)
                    y = torch.from_numpy(y_norm)

                    mat_idx = torch.full((len(X),), mat_idx_val, dtype=torch.long)
                    freq_idx = torch.zeros(len(X), dtype=torch.long)

                    with torch.no_grad():
                        preds = model(X, mat_idx, freq_idx).numpy()

                    diff_phys = (preds - y.numpy()) * stats["H_std"]
                    rmse = float(np.sqrt(np.mean(diff_phys ** 2)))

                    rms_meas = float(np.sqrt(np.mean((y.numpy() * stats["H_std"]) ** 2)))
                    rel_err = rmse / (rms_meas + 1e-12) * 100.0

                    rows.append({
                        "material": material,
                        "segment": seg_idx,
                        "n_windows": len(X_norm),
                        "rmse": rmse,
                        "rel_err": rel_err,
                    })
            else:
                print(f"  Unsupported dataset shape at root: {shape} — skipping")

    return rows




# ============================================================
# MAIN
# ============================================================

def main():
    set_seed(42)

    # 1) global stats
    stats = compute_global_normalisation(TRAIN_ROOT, TRAIN_MATERIALS, FREQS)

    # 2) load global model with embeddings
    model = GlobalLSTM(
        n_materials=len(TRAIN_MATERIALS),
        n_freqs=len(FREQS),
        input_dim=3,
        hidden_dim=32,
        mat_emb_dim=4,
        freq_emb_dim=2,
        num_layers=1,
    ).to(DEVICE)

    state = torch.load(MODEL_PATH, map_location=DEVICE)
    model.load_state_dict(state)
    model.eval()
    print("\nModel loaded.")

    # 3) run pretest
    all_rows = []
    for mat in tqdm(PRETEST_MATERIALS, desc="PRETEST materials"):
        rows = evaluate_pretest_material(mat, model, stats)
        all_rows.extend(rows)

    if not all_rows:
        print("\n⚠️ No results collected — check paths!")
        return

    df = pd.DataFrame(all_rows)

    print("\n================ PRETEST RESULTS (per segment) ================")
    print(df.to_string(index=False, float_format=lambda x: f"{x:.6f}"))

    summary = (
        df.groupby("material")[["rmse", "rel_err"]]
        .mean()
        .reset_index()
        .sort_values("material")
    )

    print("\n================ PRETEST SUMMARY (avg per material) ===========")
    print(summary.to_string(index=False, float_format=lambda x: f"{x:.6f}"))

    df.to_csv("pretest_segment_results.csv", index=False)
    summary.to_csv("pretest_material_summary.csv", index=False)

    print("\nSaved:")
    print("  pretest_segment_results.csv")
    print("  pretest_material_summary.csv")


if __name__ == "__main__":
    main()


Device: cpu

📌 Computing global normalisation stats from training CSVs...


Materials: 100%|██████████| 10/10 [10:50<00:00, 65.03s/it]
/var/folders/j_/hfnx7c8d0rs6p3gngpfmv2x00000gp/T/ipykernel_89424/1871835115.py:412: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experime


Global stats:
  B_mean: 0.000859
  B_std: 0.056539
  H_mean: -0.870553
  H_std: 15.358889
  dB_mean: 0.000006
  dB_std: 0.001092

Model loaded.


PRETEST materials:   0%|          | 0/11 [00:00<?, ?it/s]


=== Evaluating PRETEST for 3C90 ===
Found groups: []
   Using ROOT datasets: B='B_seq', H='H_seq'


...


=== Evaluating PRETEST for N30 ===
Found groups: []
   Using ROOT datasets: B='B_seq', H='H_seq'


PRETEST materials:  73%|███████▎  | 8/11 [06:07<02:19, 46.40s/it]


=== Evaluating PRETEST for N49 ===
Found groups: []
   Using ROOT datasets: B='B_seq', H='H_seq'


PRETEST materials: 100%|██████████| 11/11 [06:42<00:00, 36.58s/it]


[N87] Missing Testing_True.h5 — skipping
[N87-2] Missing Testing_True.h5 — skipping

================ PRETEST RESULTS (per segment) ================
material  segment  n_windows       rmse   rel_err
    3C90        0        184   0.202618  1.925585
    3C90        1        184   0.221436  0.580481
    3C90        2        184   2.636753  4.693029
    3C90        3        184   0.417176  2.966346
    3C90        4        184   0.222296  1.451799
    3C90        5        184   0.425891  1.142859
    3C90        6        184   0.236243  1.737835
    3C90        7        184   0.437414  1.311357
    3C90        8        184   0.529761  4.285198
    3C90        9        184   0.220200  0.621665
    3C90       10        184   0.598876  3.291909
    3C90       11        184   0.328540  2.021808
    3C90       12        184   0.268491  1.858518
    3C90       13        184  73.945435 64.742585
    3C90       14        184   0.216448  1.461226
    3C90       15        184   0.428263  1.421500
 